In [2]:
import os
import cv2
import numpy as np
from paddleocr import PPStructure,save_structure_res, PaddleOCR, draw_structure_result
from paddle.utils import try_import
from PIL import Image
from paddleocr.ppstructure.recovery.recovery_to_doc import sorted_layout_boxes, convert_info_docx

ocr_engine = PPStructure(recovery=True, structure_version='PP-StructureV2')

save_folder = './dev_v1'
pdf_path = '../data/basic.pdf'
font_path = '../fonts/simfang.ttf' # PaddleOCR下提供字体包


fitz = try_import("fitz")
imgs = []
with fitz.open(pdf_path) as pdf:
    for pg in range(0, pdf.page_count):
        page = pdf[pg]
        mat = fitz.Matrix(2, 2) # 将 PDF 页面缩放两倍，为了提高图像的分辨率和清晰度。
        pm = page.get_pixmap(matrix=mat, alpha=False)

        # if width or height > 2000 pixels, don't enlarge the image
        if pm.width > 2000 or pm.height > 2000:
            pm = page.get_pixmap(matrix=fitz.Matrix(1, 1), alpha=False)

        img = Image.frombytes("RGB", [pm.width, pm.height], pm.samples) # 将像素图转换为 PIL 图像对象。
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR) # 将 PIL 图像转换为 NumPy 数组，并从 RGB 格式转换为 BGR 格式，以便 OpenCV 使用。
        imgs.append(img)



# for index, img in enumerate(imgs):
result = ocr_engine(imgs[0])
save_structure_res(result, save_folder, os.path.basename(pdf_path).split('.')[0], 0)
h, w, _ = img.shape
res = sorted_layout_boxes(result, w)
convert_info_docx(img, res, save_folder, os.path.basename(pdf_path).split('.')[0])

[2024/06/05 14:14:42] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/liangzhu/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/liangzhu/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, 

In [3]:
for index, img in enumerate(imgs):
    result = ocr_engine(img)
    save_structure_res(result, save_folder, os.path.basename(pdf_path).split('.')[0], index)
    h, w, _ = img.shape
    res = sorted_layout_boxes(result, w)
    convert_info_docx(img, res, save_folder, os.path.basename(pdf_path).split('.')[0])

    # 保存每一页的结构化结果图片
    im_show = draw_structure_result(Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)), result, font_path=font_path)
    im_show = Image.fromarray(im_show)
    im_show.save(os.path.join(save_folder, f'result_page_{index + 1}.jpg'))

[2024/06/05 14:15:13] ppocr DEBUG: dt_boxes num : 5, elapsed : 0.6765964031219482
[2024/06/05 14:15:14] ppocr DEBUG: rec_res num  : 5, elapsed : 1.5481441020965576
[2024/06/05 14:15:15] ppocr DEBUG: dt_boxes num : 9, elapsed : 0.6778621673583984
[2024/06/05 14:15:17] ppocr DEBUG: rec_res num  : 9, elapsed : 2.485006093978882
[2024/06/05 14:15:18] ppocr DEBUG: dt_boxes num : 3, elapsed : 0.6048650741577148
[2024/06/05 14:15:19] ppocr DEBUG: rec_res num  : 3, elapsed : 0.775745153427124
[2024/06/05 14:15:19] ppocr DEBUG: dt_boxes num : 5, elapsed : 0.6725292205810547
[2024/06/05 14:15:21] ppocr DEBUG: rec_res num  : 5, elapsed : 1.320040225982666
[2024/06/05 14:15:21] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.592108964920044
[2024/06/05 14:15:22] ppocr DEBUG: rec_res num  : 2, elapsed : 0.5774540901184082
[2024/06/05 14:15:22] ppocr DEBUG: dt_boxes num : 5, elapsed : 0.6012790203094482
[2024/06/05 14:15:24] ppocr DEBUG: rec_res num  : 5, elapsed : 1.3691051006317139
[2024/06/05 14:15:24